## 1-First we import the data and do the cleaning

In [1]:
import pandas as pd
import numpy as np

### I copy-past ed the data in excel TBH

In [2]:
df = pd.read_excel('C:\\Users\\Nobody\\Documents\\Canada.xlsx') #The page that had this dataset is: 
df.head() #https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### The rows with the "Not assigned" Borough are discarded

In [3]:
df=df[df['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### The data frame was grouped in Postcode and Borough, the index was reset and the Neighbourhood columns was formated

In [4]:
df['Neighbourhood'] = df['Neighbourhood']+', '
df=df.groupby(['Postcode','Borough']).sum()

In [5]:
df.reset_index(inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern,"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union,"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill,"
3,M1G,Scarborough,"Woburn,"
4,M1H,Scarborough,"Cedarbrae,"


In [6]:
for x in range(len(df['Neighbourhood'])):
    df.loc[x,'Neighbourhood'] = df.loc[x,'Neighbourhood'][:len(df.loc[x,'Neighbourhood'])-2]
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Finally the shape is given

In [7]:
df.shape

(103, 3)

# 2- The coordinates for each postcode must be given

### I need to instal this

In [10]:
! pip install geocoder

In [14]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
z=0
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M1B'))
  lat_lng_coords = g.latlng
  z=+1
  if(z%100 == 0):
    print (z)
    
lat_lng_coords

KeyboardInterrupt: 

### The geocoder is not working so i will use the option B

In [8]:
df2 = pd.read_csv('C:\\Users\\Nobody\\Documents\\Geospatial_Coordinates.csv')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df.sort_values(['Postcode'])
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df2.sort_values(['Postal Code'])
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Here the job is done

In [30]:
df['Latitude']=df2['Latitude']
df['Longitude']=df2['Longitude']
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitud,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,-79.239476


# 3- Actually clustering

In [26]:
!pip install folium
import folium

In [49]:
la = df['Latitude'].mean()
lo = df['Longitude'].mean()

## Here are the Postcodes with their neighbourhoods

In [85]:
m = folium.Map(location=[la,lo], zoom_start=11)
mthings=folium.map.FeatureGroup()
for n in range(len(df['Latitude'])):
    La= df.loc[n,'Latitude']
    Lo= df.loc[n,'Longitude']
    pop = df.loc[n,'Postcode'] + ': ' + df.loc[n,'Neighbourhood']
    mthings.add_child(folium.Marker([La,Lo],popup=pop))
m.add_child(mthings)
m

## Now i want to make a density clustering

In [150]:
from sklearn.cluster import DBSCAN
db=DBSCAN(eps=.02,min_samples=3).fit(df[['Latitude','Longitude']])
df['Group'] = db.labels_
df['Group'] = df['Group'] + 1
db.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1,
       -1, -1, -1, -1, -1, -1,  2,  2,  3,  3,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, -1,  2,  2,  2,  2,  2,
        2,  2,  2, -1, -1,  2, -1,  2,  2, -1, -1,  1, -1, -1, -1, -1, -1,
        2, -1,  3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1], dtype=int64)

In [162]:
colors = ['black', 'blue',
           'darkred', 'green', 
          'orange', 'pink', 'purple', 'red', 'white']
mm = folium.Map(location=[la,lo], zoom_start=11)
mmthings=folium.map.FeatureGroup()
for n in range(len(df['Latitude'])):
    La= df.loc[n,'Latitude']
    Lo= df.loc[n,'Longitude']
    pop = df.loc[n,'Postcode'] + ': ' + df.loc[n,'Neighbourhood']
    Dcolor = colors[df.loc[n,'Group']]
    mmthings.add_child(folium.CircleMarker([La,Lo],popup=pop,color=Dcolor, fill_color=Dcolor))
mm.add_child(mmthings)
mm

## The place where i lived (according to the course) was heavily populated? Because i want to live in a similar place

#### This data was in https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Table.cfm?Lang=Eng&T=1201&SR=1&S=22&O=A&RPP=9999&PR=0

In [143]:
population = pd.read_csv('C:\\Users\\Nobody\\Documents\\Cpopulation.csv')
population.head()

,Postcode,Population
0,A0A,46587
1,A0B,19792
2,A0C,12587
3,A0E,22294
4,A0G,35266


In [144]:
population.set_index('Postcode',inplace=True)

In [145]:
population = population.loc[df['Postcode'].values[:]]
population.reset_index(inplace=True)
population.sort_values('Postcode')
df['Population']=population['Population']
df.head()

C:\Users\Nobody\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood,Latitude,Longitud,Longitude,Group,Population
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,-79.194353,0,66108.0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,-79.160497,0,35626.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,-79.188711,0,46943.0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,-79.216917,0,29690.0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,-79.239476,0,24383.0


### Here i will erase the outliers

In [173]:
Dd=df.groupby('Group').mean()
aux=df.groupby('Group').sum()
Dd['Population'] = aux['Population']
Dd=Dd[1:5]
Dd.reset_index(inplace=True)
Dd

,Group,Latitude,Longitud,Longitude,Population
0,1,43.770644,-79.405678,-79.405678,116060.0
1,2,43.727089,-79.497572,-79.497572,62001.0
2,3,43.671339,-79.388962,-79.388962,572565.0
3,4,43.663756,-79.326018,-79.326018,57339.0


### The next map give us the center of the clusters with their population (excluding the outliers)

In [178]:
mmm = folium.Map(location=[la,lo], zoom_start=11)
mmmthings=folium.map.FeatureGroup()
for n in range(len(Dd['Latitude'])):
    La= Dd.loc[n,'Latitude']
    Lo= Dd.loc[n,'Longitude']
    pop = Dd.loc[n,'Population']
    Dcolor = colors[Dd.loc[n,'Group']]
    mmmthings.add_child(folium.CircleMarker([La,Lo],radius=50,popup=pop,color=Dcolor, fill_color=Dcolor))
mmm.add_child(mmmthings)
mmm

### By clicking the circles one can get to know that if we lived somewhere near the left border of the red circle, then living somewhere close the border of the green or yellow circles would be OK, wouldn't it?